In [ ]:
%%capture
!pip install langchain openai datasets faiss-gpu sentence_transformers

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


#🔗 **Origins of CoT Prompting**

- CoT was introduced by Wei et al. in their 2022 paper, [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://browse.arxiv.org/pdf/2201.11903.pdf).

- It's a response to the need for better reasoning in large language models.

💡 **What is CoT Prompting?**
- CoT stands for Chain of Thought Prompting.

- It's about guiding language models through a step-by-step reasoning process.

- The model shows its reasoning, forming a "chain" of thoughts, rather than just giving an answer.

- Mimics human-like thought processes for complex tasks.


### 🎓 **Standard vs. CoT Prompting Example**

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2201.11903/assets/x2.png" alt="Image Description" style="width:100%">
  <figcaption>Examples of ⟨input, chain of thought, output⟩ triples for arithmetic, commonsense, and symbolic reasoning benchmarks. Chains of thought are highlighted.</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2201.11903">Image Source</a>
</figure>

- Standard: Direct answer with no reasoning (e.g., "11").

- CoT: Detailed reasoning steps (e.g., "Roger started with 5, added 6 from 2 cans, so 5 + 6 = 11").

### 🚀 **Usage of CoT Prompting**

1. **Enhanced Reasoning**: Breaks down complex problems into simpler steps.

2. **Combination with Few-Shot Prompting**: Uses examples to guide model responses, great for complex tasks.

3. **Interpretability**: Offers a clear view of the model's thought process.

4. **Applications**: Useful in arithmetic, commonsense reasoning, and symbolic tasks.

### ✨ **Impact of CoT Prompting**

- Improves accuracy and insightfulness in responses.

- Focuses on reasoning and interpretability, especially in complex scenarios.

The code below downloads an dataset which has over [1.8 million Chain of Thought examples](https://huggingface.co/datasets/kaist-ai/CoT-Collection).

In [ ]:
from datasets import load_dataset

dataset = load_dataset("kaist-ai/CoT-Collection")

In [ ]:
dataset['train']

Dataset({
    features: ['source', 'target', 'rationale', 'task', 'type'],
    num_rows: 1837928
})

In [ ]:
examples = dataset['train'].to_pandas()[['source', 'target', 'rationale']]

In [ ]:
examples

,source,target,rationale
0,Article: Phytochemistry is a branch of plant b...,from plants,The article states that many medicinal and rec...
1,"In this task, you are provided with an article...",Regulation,This act has all the features of Regulation. T...
2,What's the answer to that question: where did ...,Battle of Appomattox Court House,The Battle of Appomattox Court House was a bat...
3,Extract the answer to the question from the fo...,major heart attacks,The answer is the number of major heart attack...
4,"Which entity is this text about?\n\nRichard ""R...",Red Skelton,The text is about Red Skelton. The context des...
...,...,...,...
1837923,Answer the following question:\n\nthe person w...,designer,"The answer is ""designer""\nA database designer ..."
1837924,Exercise: read the text and answer the questio...,True,The text explains that bumpers are legally req...
1837925,Just minutes after Donald Trump fired Rex Till...,A,The question of what Tillerson said Trump shou...
1837926,"In this task, you're given four sentences of a...",A,"Sentence4 shows that the man is ill and sad, s..."


We'll sample just 10,000 of these Chain of Thought examples and save them as a list of dictionaries.

In [ ]:
selected_examples = examples.sample(n=10_000).to_dict(orient='records')

In [ ]:
selected_examples[0]

{'source': 'If "A man in a black coat walks past a red spaceship with a parking ticket stuck to its window.", does this mean that "A man in a blue coat walks pas a red spaceship."?\n\nOPTIONS:\n- yes\n- it is not possible to tell\n- no',
 'target': 'no',
 'rationale': 'The statement "A man in a blue coat walks pas a red spaceship" is too general and does not accurately reflect the description of the scene provided. The correct answer should instead be "a black coat", not "a blue coat". So, the answer is no.'}

For this example we'll use an open source embedding model from HuggingFace so we don't accumulate a huge bill for OpenAI

In [ ]:
%%capture

from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "BAAI/bge-base-en-v1.5"

encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

To set up a Chain of Thought prompt you will use a `FewShotPromptTemplate` as well as an example selector.

In this example you'll use `MaxMarginalRelevanceExampleSelector`, but you can use any of the example selectors you learned about before.

In [ ]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector

from langchain.vectorstores import FAISS

from langchain.prompts import FewShotPromptTemplate, PromptTemplate


In [ ]:
prefix = "Consider the following as examples of how to reason:"

examples_template = """Query: {source}

Rationale: {rationale}

Response: {target}
"""

suffix = """Using a similar reasoning approach, answer the users question which is delimited by triple backticks.

User question: ```{input}```

Take a deep breath, break down the user's query step-by-step, and provide a clear chain of thought in your response."
"""

In [ ]:
examples_prompt = PromptTemplate(
    input_variables=["source", "rationale", "target"],
    template=examples_template
)

There's a large number of examples to embed and add to the vector store. The below cell will take a few minutes to run. It took me ~3 minutes using the free T4 GPU provided on Google colab.

In [ ]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # The list of examples available to select from.
    selected_examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings,
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=7,
)

In [ ]:
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"]
)

In [ ]:
query = """Here's a short story: A lifter uses a barbell, but moves a jump rope\
in a wider arc. The object likely to travel further is (A) the jump rope (B) the\
barbell.

What is the most sensical answer between "a jump rope" and "a barbell"?
"""

In [ ]:
print(mmr_prompt.format(input=query))

Consider the following as examples of how to reason:


Query: Here's a short story: Allison and Debra are each competing in a Karate tournament. Allison is a trained athlete and extremely flexible. Debra is an amateur and is not flexible at all. Who is more likely to pull something during the high kick portion of the event? (A) Allison (B) Debra.

What is the most sensical answer between "Debra" and  "Allison"?

Rationale: The answer is "Allison" because Allison is flexible and high kicks require flexibility to perform.

Response: Allison



Query: Here's a short story: John was walking down the driveway to check the mail.  As he got closer to the mailbox, what got smaller, (A) house (B) mailbox.

What is the most sensical answer between "house" and  "mailbox"?

Rationale: The answer is house. 
Why? Because as John approaches the mailbox, he would get closer to it and therefore it appears larger from his perspective than if he were say on the other side of a room looking at the same ma

In [ ]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

llm = OpenAI()

llm_chain = LLMChain(
    llm=llm,
    prompt=mmr_prompt,
    verbose=True
)

In [ ]:
llm_chain.run({"input":query})



> Entering new LLMChain chain...
Prompt after formatting:
Consider the following as examples of how to reason:


Query: Here's a short story: Allison and Debra are each competing in a Karate tournament. Allison is a trained athlete and extremely flexible. Debra is an amateur and is not flexible at all. Who is more likely to pull something during the high kick portion of the event? (A) Allison (B) Debra.

What is the most sensical answer between "Debra" and  "Allison"?

Rationale: The answer is "Allison" because Allison is flexible and high kicks require flexibility to perform.

Response: Allison



Query: Here's a short story: John was walking down the driveway to check the mail.  As he got closer to the mailbox, what got smaller, (A) house (B) mailbox.

What is the most sensical answer between "house" and  "mailbox"?

Rationale: The answer is house. 
Why? Because as John approaches the mailbox, he would get closer to it and therefore it appears larger from his perspective than if he

'\nRationale: The answer is "jump rope" because the lifter is moving the jump rope in a wider arc than the barbell. Therefore, due to the increased distance the jump rope will travel further than the barbell.\n\nResponse: jump rope'

In [ ]:
query = """
Given the fact that: Inhaling, or breathing in, increases the size of the chest, \
 which decreases air pressure inside the lungs. Answer the question: If Mona is \
 done with a race and her chest contracts, what happens to the amount of air \
 pressure in her lungs increases or decreases?
"""

In [ ]:
llm_chain.run({"input":query})



> Entering new LLMChain chain...
Prompt after formatting:
Consider the following as examples of how to reason:


Query: Answer the question based on the following text.

Question:


Air pressure inside the lungs will increase or decrease when one exhales out deeply? 


Text:

Inhaling, or breathing in, increases the size of the chest, which decreases air pressure inside the lungs.

Rationale: Inhaling increases the size of your chest, which causes air pressure to decrease. Exhaling decreases the size of your chest and increases air pressure inside the lungs. So, when one exhales out deeply, their lung's internal pressure would increase. The answer is thus "increase".

Response: increase



Query: Given the fact that:

The longer the exposure, the more light that the telescope can gather to make the image.

Answer the question:


If Mona is an astronomer and she is looking through a telescope, what happens to the amount of light gathered by the telescope if she shortens the exposure i

'\nRationale: The user\'s query states that inhaling increases the size of the chest, which decreases air pressure inside the lungs. So if Mona is done with a race and her chest contracts, her inhalation will decrease, causing her chest size to decrease as well. This in turn will lead to an increase in air pressure in her lungs. Therefore, the answer should be "increases".\n\nResponse: increases'